In [3]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

In [25]:
class ClassToken(Layer):
    def __init__(self):
        super().__init__()

    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value = w_init(shape=(1, 1, input_shape[-1]), dtype=tf.float32),
            trainable = True
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        hidden_dim = self.w.shape[-1]
        print(batch_size, hidden_dim)

        cls = tf.broadcast_to(self.w, [batch_size, 1, hidden_dim])
        cls = tf.cast(cls, dtype=inputs.dtype)
        return cls

In [28]:
def mlp(x, cf):
    x = Dense(cf["mlp_dim"], activation="gelu")(x)
    x = Dropout(cf["dropout_rate"])(x)
    x = Dense(cf["hidden_dim"])(x)
    x = Dropout(cf["dropout_rate"])(x)

    return x

In [29]:
def transformer_encoder(input, cf):
    skip_1 = input
    x = LayerNormalization()(input)
    x = MultiHeadAttention(
        num_heads=cf["num_heads"], key_dim=cf["hidden_dim"]
    )(x, x)
    x = Add()([x, skip_1])

    skip_2 = x
    x = LayerNormalization()(x)
    x = mlp(x, cf)
    x = Add()([x, skip_2])
    
    return x

In [37]:
def ViT(cf):
    
    """ Inputs """
    input_shape = (cf["num_patches"], cf["patch_size"] * cf["patch_size"] * cf["num_channels"])
    inputs = Input(input_shape)
    print(inputs.shape)

    """ Patch + Position Embedding """
    patch_embed = Dense(cf["hidden_dim"])(inputs)

    positions = tf.range(start=0, limit=cf["num_patches"], delta=1)
    #print(positions)

    position_embed = Embedding(input_dim=cf["num_patches"], output_dim=cf["hidden_dim"])(positions)
    print(position_embed.shape)

    embed = patch_embed + position_embed
    print(embed.shape)

    """ Adding Class Token """
    token = ClassToken()(embed)
    x = Concatenate(axis=1)([token, embed])
    print(x.shape)

    """ Transformer Encoder """
    for _ in range(cf["num_layers"]):
        x = transformer_encoder(x, cf)

    #print(x)

    """ Classificatin Head """
    x = LayerNormalization()(x)
    x = x[:, 0, :]
    print(x.shape)
    x = Dropout(0.1)(x)
    x =Dense(cf["num_classes"], activation="softmax")(x)

    model = Model(inputs, x)

    return model

In [38]:
if __name__ == "__main__":
    
    config = {}
    config["num_layers"] = 12
    config["hidden_dim"] = 768
    config["mlp_dim"] = 3072
    config["num_heads"] = 12
    config["dropout_rate"] = 0.1
    config["num_patches"] = 256
    config["patch_size"] = 32
    config["num_channels"] = 3
    config["num_classes"] = 10 # Depend on the problem this will change

    model = ViT(config)
    model.summary()

(None, 256, 3072)
(256, 768)
(None, 256, 768)
Tensor("class_token_6/strided_slice:0", shape=(), dtype=int32) 768
(None, 257, 768)
(None, 768)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_15 (InputLayer)       [(None, 256, 3072)]          0         []                            
                                                                                                  
 dense_109 (Dense)           (None, 256, 768)             2360064   ['input_15[0][0]']            
                                                                                                  
 tf.__operators__.add_7 (TF  (None, 256, 768)             0         ['dense_109[0][0]']           
 OpLambda)                                                                                        
                                                   